# 🛠️ 02 - Özellik Mühendisliği (Feature Engineering)

Bu notebook içinde model performansını artırmak için, mevcut verilerden anlamlı ve etkili yeni özellikler türetilecektir.

- Zaman serisi bazlı gecikmeli (lag) değişkenler
- Rolling mean (hareketli ortalama)
- İlçe bazlı özellikler


In [2]:
import pandas as pd

# combined_df dosyasını bir önceki defterden dışa aktardıysan buradan yükle
# Örneğin: combined_df.to_csv("../data/combined_df.csv", index=False)

combined_df = pd.read_csv("../data/combined_df.csv", parse_dates=["tarih"])

# Kontrol
combined_df.head()


,tarih,ilce,bildirimsiz_sum,bildirimli_sum,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,tatil_mi,yil,ay,gun,hafta,haftanin_gunu
0,2021-01-01,izmir-aliaga,5.0,0,13.275000,55.045833,73.020833,85.991667,167.204167,4.266667,1.000000,13.929167,1,2021,1,1,53,4
1,2021-01-02,izmir-aliaga,13.0,0,12.870833,36.125000,92.858333,82.137500,106.650000,2.891667,1.000000,13.645833,0,2021,1,2,53,5
2,2021-01-03,izmir-aliaga,4.0,0,12.633333,64.870833,39.266667,79.120833,122.520833,3.437500,8.429167,12.845833,0,2021,1,3,53,6
3,2021-01-04,izmir-aliaga,9.0,0,13.733333,47.783333,78.945833,70.312500,123.408333,4.608333,1.275000,13.775000,0,2021,1,4,1,0
4,2021-01-05,izmir-aliaga,2.0,0,14.375000,30.379167,95.716667,75.220833,195.720833,3.991667,10.720833,14.900000,0,2021,1,5,1,1


In [3]:
# Önce sıralama
combined_df = combined_df.sort_values(by=["ilce", "tarih"])

# Lag sütunları üret
for lag in [1, 2, 3]:
    combined_df[f"lag_{lag}"] = combined_df.groupby("ilce")["bildirimsiz_sum"].shift(lag)

# Kontrol
combined_df[["ilce", "tarih", "bildirimsiz_sum", "lag_1", "lag_2", "lag_3"]].head(10)


,ilce,tarih,bildirimsiz_sum,lag_1,lag_2,lag_3
0,izmir-aliaga,2021-01-01,5.0,NaN,NaN,NaN
1,izmir-aliaga,2021-01-02,13.0,5.0,NaN,NaN
2,izmir-aliaga,2021-01-03,4.0,13.0,5.0,NaN
3,izmir-aliaga,2021-01-04,9.0,4.0,13.0,5.0
4,izmir-aliaga,2021-01-05,2.0,9.0,4.0,13.0
5,izmir-aliaga,2021-01-06,2.0,2.0,9.0,4.0
6,izmir-aliaga,2021-01-07,3.0,2.0,2.0,9.0
7,izmir-aliaga,2021-01-08,5.0,3.0,2.0,2.0
8,izmir-aliaga,2021-01-09,8.0,5.0,3.0,2.0
9,izmir-aliaga,2021-01-10,8.0,8.0,5.0,3.0


In [4]:
# Hareketli ortalama (rolling mean) sütunları
for window in [3, 7]:
    combined_df[f"rolling_mean_{window}"] = (
        combined_df
        .groupby("ilce")["bildirimsiz_sum"]
        .shift(1)  # bugünün verisi dahil olmasın!
        .rolling(window=window, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )

# Kontrol
combined_df[[
    "ilce", "tarih", "bildirimsiz_sum", "lag_1", "rolling_mean_3", "rolling_mean_7"
]].head(10)


,ilce,tarih,bildirimsiz_sum,lag_1,rolling_mean_3,rolling_mean_7
0,izmir-aliaga,2021-01-01,5.0,NaN,NaN,NaN
1,izmir-aliaga,2021-01-02,13.0,5.0,5.000000,5.000000
2,izmir-aliaga,2021-01-03,4.0,13.0,9.000000,9.000000
3,izmir-aliaga,2021-01-04,9.0,4.0,7.333333,7.333333
4,izmir-aliaga,2021-01-05,2.0,9.0,8.666667,7.750000
5,izmir-aliaga,2021-01-06,2.0,2.0,5.000000,6.600000
6,izmir-aliaga,2021-01-07,3.0,2.0,4.333333,5.833333
7,izmir-aliaga,2021-01-08,5.0,3.0,2.333333,5.428571
8,izmir-aliaga,2021-01-09,8.0,5.0,3.333333,5.428571
9,izmir-aliaga,2021-01-10,8.0,8.0,5.333333,4.714286


In [5]:
# İşlenmiş veriyi kaydet
combined_df.to_csv("../data/processed_df.csv", index=False)
